In [1]:
from model.GP_model import GP
import gpjax as gpx
import numpy as np
from matplotlib import pyplot as plt
import jax.numpy as jnp
from math import ceil

/home/lolanff/plant-rl/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/lolanff/plant-rl/.venv/lib/python3.12/site-packages/cola/backends/backends.py:75: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(cls, tree_flatten, tree_unflatten)


In [2]:
data_type = "mean"
time_step = "10min"
action_trace = 0  # choose from [0, 1, 3, 5, 7, 9] where 0 corresponds to no trace and larger value means longer-term averaging

In [3]:
# Retrieve cleaned data
data = np.load(f"./GP_data/{data_type}_size_{time_step}.npy")
print(f"Number of data points: {len(data)}")

if time_step == "1day":
    input_data = data[
        :,
        [
            1,
            3 * ceil(action_trace / 2) + 2,
            3 * ceil(action_trace / 2) + 3,
            3 * ceil(action_trace / 2) + 4,
        ],
    ]  # input = [morning area, one-hot action trace of red/white/blue]
elif time_step == "10min":
    input_data = data[
        :, [0, 1, ceil(action_trace / 2) + 2]
    ]  # input = [time step, area, action trace of intensity (bright=1)]

output_data = data[:, -1].reshape(-1, 1)  # output = [next area]

Number of data points: 7592


In [5]:
GP_model = GP(input_data, output_data)  # , kernel = gpx.kernels.RBF())

TypeError: Cannot determine dtype of Traced<ShapedArray(float64[])>with<DynamicJaxprTrace(level=1/0)>

In [ ]:
import pickle

with open(
    f"./models/{data_type}_size_{time_step}_trace{action_trace}.pickle", "wb"
) as f:
    pickle.dump(GP_model, f)